<a href="https://colab.research.google.com/github/arturolopm/ML/blob/main/RNNPlaygenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%tensorflow_version 2.x # this line es not required inell you are in a notebook

from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x # this line es not required inell you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


Dataset

In [9]:
path_to_file = tf.keras.utils.get_file('MMsatan.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [10]:
from google.colab import files #import nfiles from your pc
path_to_file = list(files.upload().keys())[0]

Saving Lady Masacre by Mario Mendoza (z-lib.org).epub.txt to Lady Masacre by Mario Mendoza (z-lib.org).epub.txt


In [11]:
#read, thenmmm decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the numbers of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 373281 characters


In [13]:
#Take a look of the forst 150 characters in text
print(text[5000:5500])

que solía dormir en posición fetal y que en varios de mis sueños aparecía mi madre (ya fallecida), siempre bondadosa y cariñosa, acariciándome el pelo, diciéndome cuánto me quería, abrazándome antes de salir para el colegio. Definitivamente la niñez no es un estado superado, sino una dimensión de la conciencia que está ahí, latente, y que se activa por momentos para que nos quede claro que seguimos siendo esos seres diminutos, frágiles y lúdicos que tanto disfrutaban de la protección de sus adul


In [14]:
vocab = sorted(set(text))
#creating a apmping for unique characters to indeces
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int= text_to_int(text)

In [15]:
#lets look at how part of our text is encoded
print("text: ", text[:14])
print("Encoded: ", text_to_int(text[:14]))

text:  La vida de Fra
Encoded:  [33 48  1 69 56 51 48  1 51 52  1 27 65 48]


In [16]:
#convert nuermic values to text
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:14]))

La vida de Fra


Create training expamples
input: hell | output: ello

In [17]:
#create a strea mof chmaramcters from oumr data
seq_length = 100 # leght of sequence for a training example
expamples_per_epoch =len(text)//(seq_length+1)
 
 #create training examples / targets

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next use batch metod mto turn this stream of characters inmto batches of desired length


In [18]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

now use these sequences of length 101 and split them into input and output

In [19]:
def split_input_target(chunk): # for the example : hello
    input_text= chunk[:-1] # hell
    target_text = chunk[1:] # ello
    return input_text, target_text # hell, ello

dataset = sequences.map(split_input_target) # we use map to apply the function avobe to every entry

In [20]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
La vida de Frank Molina, periodista especializado en judiciales, da un giro inesperado el día que de

OUTPUT
a vida de Frank Molina, periodista especializado en judiciales, da un giro inesperado el día que dec


EXAMPLE

INPUT
ide abrir una oficina y empezar a trabajar como detective privado. A sus manos llega el caso de un e

OUTPUT
de abrir una oficina y empezar a trabajar como detective privado. A sus manos llega el caso de un ex


finally make training batches

In [21]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # vocab is numbmer of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

#Buffer size to shuffle the dataset
#(TF data is designed to work posibbly with infinite sequences
# so it doesn't attempt to shuffñe the entire sequence in memory. Instead
# it maintains a buffer inm wich it shuffles elements).
BUFFER_SIZE = 10000
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Build the *model*, we will our use an embedding layer a LTSM and ine dense layer that contains a node for each unique character in our trinig data, the layer will give us a probability distribution over all nodes

In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                                         batch_input_shape=[batch_size, None]),
                               tf.keras.layers.LSTM(rnn_units,
                                                    return_sequences=True,
                                                    stateful=True,
                                                    recurrent_initializer='glorot_uniform'),
                               tf.keras.layers.Dense(vocab_size)
  ])
  return model 

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23296     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 91)            93275     
Total params: 5,363,547
Trainable params: 5,363,547
Non-trainable params: 0
_________________________________________________________________


Create loss function
but first predictions prior training

In [23]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) # ask our model for a prediction on our first batch of training data
  print(example_batch_predictions.shape, "# (batch size, sequence_length, vocab_size)") #print out the output shape

(64, 100, 91) # (batch size, sequence_length, vocab_size)


In [24]:
#we can see that the predoction is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.1671145e-03 -8.3997729e-05  7.3874439e-03 ...  5.7104309e-03
    2.7243162e-03 -2.0176182e-03]
  [-8.1098219e-04 -2.1475989e-03  4.7739418e-03 ...  8.9015055e-04
    2.8375657e-03 -4.8945681e-03]
  [ 1.7900246e-03  6.3053169e-04  2.1636565e-03 ... -1.6084180e-04
    1.1616480e-03 -5.9067816e-03]
  ...
  [ 1.4718919e-03  8.5621709e-03  1.0628840e-03 ...  4.9963454e-03
    1.6154278e-02  6.4539639e-03]
  [ 3.5397420e-03  5.0443881e-03  4.3546373e-04 ...  1.6358262e-04
    1.6398212e-02  1.0683674e-02]
  [ 2.1511510e-03  2.5350747e-03 -9.1433985e-04 ...  3.8776253e-03
    1.4329702e-02  2.4245176e-03]]

 [[ 1.1160151e-03 -3.1310152e-03 -3.4986972e-04 ... -5.9874328e-03
    2.5330242e-04  9.4693210e-03]
  [ 3.2916856e-03 -3.6639879e-03 -2.1014193e-03 ... -8.1586177e-03
    5.6619137e-03  1.2093884e-02]
  [ 2.4805539e-03 -3.2341997e-03 -2.7923912e-03 ...  1.8480213e-03
    1.2666127e-03  4.3316642e-03]
  ...
  [-1.6188405e-02  8.0116547e-04  4.8368219e-03 ...  1.0398675e

In [25]:
#lets examine 1 prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-2.1671145e-03 -8.3997729e-05  7.3874439e-03 ...  5.7104309e-03
   2.7243162e-03 -2.0176182e-03]
 [-8.1098219e-04 -2.1475989e-03  4.7739418e-03 ...  8.9015055e-04
   2.8375657e-03 -4.8945681e-03]
 [ 1.7900246e-03  6.3053169e-04  2.1636565e-03 ... -1.6084180e-04
   1.1616480e-03 -5.9067816e-03]
 ...
 [ 1.4718919e-03  8.5621709e-03  1.0628840e-03 ...  4.9963454e-03
   1.6154278e-02  6.4539639e-03]
 [ 3.5397420e-03  5.0443881e-03  4.3546373e-04 ...  1.6358262e-04
   1.6398212e-02  1.0683674e-02]
 [ 2.1511510e-03  2.5350747e-03 -9.1433985e-04 ...  3.8776253e-03
   1.4329702e-02  2.4245176e-03]], shape=(100, 91), dtype=float32)


In [26]:
#and finally we'll look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
#it is 65 bc we have 65 characters, each value is the probability of each character ocurring next

91
tf.Tensor(
[-2.1671145e-03 -8.3997729e-05  7.3874439e-03  4.7776392e-03
 -2.1570111e-03 -3.1397783e-03  3.9623759e-04 -5.3829215e-03
 -1.3799104e-03  2.1133812e-03 -3.0295779e-03  5.3933989e-03
 -3.3961888e-04  4.4129188e-03  9.4672036e-04 -4.8577201e-04
  5.4175453e-04  2.0725611e-03 -3.8496614e-03 -5.5133458e-04
  6.2935073e-03 -2.4465765e-03 -1.3631160e-03 -1.5971363e-03
  5.2285860e-03 -5.7106209e-03 -1.2696375e-03 -3.1692511e-04
  2.9519682e-03 -3.8154968e-03  4.9808377e-04  6.4242573e-05
 -6.5666466e-04 -1.5951490e-03 -4.4140425e-03 -5.9380592e-03
 -3.6029774e-03 -3.4411799e-03 -2.1477044e-03  4.5031835e-03
 -3.1991233e-03  2.8912115e-03 -2.3668626e-04  9.3727442e-04
  3.9253281e-03  2.6101095e-03  3.4115498e-03  5.8651953e-03
  1.5760981e-04 -2.5889431e-03 -4.7512446e-03 -4.6944933e-04
 -3.0846084e-03  4.2494875e-03  2.2040070e-03 -2.4355652e-03
  5.0942441e-03  2.3762076e-03 -3.6227605e-03  5.4271257e-04
 -3.4038052e-03  2.7798996e-03  1.7889736e-03 -5.1471209e-03
 -6.557531

In [27]:
#if we want to determine the preicte character we need to sample the outout distribution (pick a value base on its probability)
sampled_indices = tf.random.categorical(pred, num_samples=1)

#now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices,(1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars #and this is what the model predicted for training sequence 1

'oRzónET7QcBgI…)9( ÍÉ«0«BJ¿h9?YXaG;«AKdfJbBVivú8-PA…ÁéGx;g0CNüaOADÁiuérCEuPünv?hxjGk 0d)3m!qdqíf)fYÉt'

now define a function  to compare output to expected aoutput and give us a numeric value reoresenting how close they are

In [28]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

compiling model
innthis point we can think our problem as a classification problemm where the model predicts the probability of each unique letter comming next

In [29]:
model.compile(optimizer='adam', loss= loss)

Creating checkpoints 
now we are configure our odel to save chackpoint as it trains. This will allow is to load our model from a checkpoint and continue training it

In [30]:
#Directory where the checkpoint will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_MM(epoch)")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only= True)

Training
Finally we will start training the model.


In [31]:
history = model.fit(data, epochs =48, callbacks=[checkpoint_callback])

Epoch 1/48
 3/57 [>.............................] - ETA: 5:45 - loss: 4.2311

KeyboardInterrupt: ignored

loading the model
We'll rebuiñl the model frim a checkpoint using a batch size of 1 so that we can feed one ppiece of text ti the model and hve it make a prediction

In [114]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

once it is finished training we can find the latest checkpoint that stores the models weight using following line.


In [115]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load any checkpoint we want by specifying the exact file to load.


In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_"+ str(checkpoint_num)))
model.build(tf.TensorShape([1,None]))

Genereating text
now we can use thw tensorflow provided function to generate some text any string we'd like.

In [116]:
def generate_text(model, start_string):
  #Evaluation step, generates text using the learned model

  #number of characters to generate
  num_generate = 1000

  # converting our start tring to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  #Empty sting to store the results
  text_generated = []
  #low temperatures results in more predictable text.
  #Higher temperatures results in mmore surrprising text-
  #experiment to find the best setting
  temperature = 1.0

  #here the batch size == 1
  model.reset_states()

  for i in range(num_generate):
    predictions = model(input_eval)
    #remove the batch dimentions
    predictions = tf.squeeze(predictions, 0)

    #using a categorical distribution to predict the character predicted by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    #We pass the predicted character as the next input to the model
    # along with the previous hidden state

    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [119]:
inp = input("Type a string: ")
print(generate_text(model,inp))

Type a string: Hola, como estas?
Hola, como estas?F0Z££îÉ<èêè’V8F£xR~ïÉÉÉêF<èëç~ÉFWëëWF’;~ï4FèèÔwè8x\(ZçZ<èùèèZ;/'/(à\ëùèF8Ôë(îûèîîOï4ïW£Zè~Ôè0É;£îF’;'ÉèwÔèè£ÔèZÉZG’’~XëOÎè<ë£î£îÉàZÔ»GGÔ7’è\FF<ç~îà\F~È£FÉèàï[ÉêÉèèèï8F"Z<«0èZR8ZW4'ÂïÉFÔ2Zx<WOàïF~('ZwÔïWç'~F0ïÂèZ'\0ÉFÔ8ùÈèè/kFêùFë~XZëÈ’Fç<X~(ïFÉÉèîF<OU£ï\WfF0Z;'F<è~îF9ÈÎ<ÈÂÂî<WÔÉà^«ÎïF£M'RZùLG\'É»ùw<èFêZçêèô<zî\/RL£û(ëëW(èèZÂù4ïOOZÈÔ[0J»É;'îwïÉHF7<£ù’~É£ïîZ<WÔà~É;ïZ£à0ïç<'WÔOàèï/FûÈèèîWïOÂ'OïFàÉIèç<ZFà;ÉïO<èèàwïîF0ï0ù/Éè"ïG’èFèè<£îG6xùëëZïZ'TWùèêZ!Hù'ùàêàZ£''ï^ÉÉèZ<èÔçèè’;8ïÈFÉÎ'àOè"èX£GÉÉ(Ô~ï£î<Uîè»~îÈç~Zè\5ÉèîF'àFç»ÉçëÉàèX'6àçxïWÂëZ<'àOà£FEùèë£ïÔéàèèïè\É4èèÈ~èèèèkÔFÉÈÂïxWèÈ<ZXJÂèàW6èêÉèïHFèÉïW£îF;îGFçW'0F[£’ZFëïë8~[8ïx0Ô£ZRà(êOFêè(ëï’£GFw'ïRîZÉïwX»’'Éù£\ïîÉ’[ïî<ïÈÉJ8ÈRÔçÉ[èà(’F'ëèZFZÉ£èÔFèR'Oà£èù£OF6<ÉFê(£Â'F)»ïîîèF/F<èèHFçïÈFZÉèUç5ï£GîÉèé£ÈC£ùFFFxFïçFÉçî'à~~ÎùàùMXïù'’FFùçWëZ<à8ZÉZîÉè»àëFÉèÎ''’B[êàGÉçÎÈàôOêÈ:<àà’FÈFÉï8(ëG"îÉïZ-ëÂÉùFIÉèèèÂï;W0ïùÈèàç£üÈ\<è<ïÔ:ZFÉèèàPZFïZùJÉÉ/~(îïà0UèÂXù~ZÉ'kÉç(ïW<ï'à0~3<î5Fx8ÔÈXêèFïZëî(FèFà\~î'ïÔÉÉàùZÉç'